In [ ]:
import numpy as np
import torch
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt
from google.colab import drive

# load Google Drive
drive.mount('/content/drive')

# Mixup function
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# load image
def load_image(image_path, enhance_contrast=False):
    img = Image.open(image_path).convert('RGB')  # Open the image and make sure it is in RGB format

    # Contrast enhancement
    if enhance_contrast:
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2.0)  # Enhance contrast (adjustable parameter, 2.0 means 2 times Augmented)

    img = img.resize((224, 224))  # Resize the image for easier visualization
    img_array = np.asarray(img) / 255.0  # Normalized to [0, 1]
    return img_array

# Convert an image to a PyTorch Tensor
def image_to_tensor(image_array):
    return torch.tensor(image_array).permute(2, 0, 1).unsqueeze(0)  # (H, W, C) -> (1, C, H, W)

# Convert PyTorch Tensor back to image
def tensor_to_image(tensor):
    tensor = tensor.squeeze(0).permute(1, 2, 0).detach().cpu().numpy()  # (1, C, H, W) -> (H, W, C)
    return np.clip(tensor, 0, 1)  # Ensure that the value is in the range [0, 1]

# Main function: Visualizing Mixup
def visualize_mixup(image_path):
    # Loading and processing images
    img1 = load_image(image_path)  # original picture
    img2 = load_image(image_path, enhance_contrast=True)  # Enhanced contrast image

    # Convert image to Tensor
    img1_tensor = image_to_tensor(img1)
    img2_tensor = image_to_tensor(img2)

    # Create a virtual label (no label is shown in this example)
    y1 = torch.tensor([1])
    y2 = torch.tensor([0])

    # Apply Mixup
    mixed_img_tensor, _, _, lam = mixup_data(
        torch.cat([img1_tensor, img2_tensor], dim=0),
        torch.cat([y1, y2], dim=0),
        alpha=0.4,
        use_cuda=False
    )

    # Convert the Tensor back to an image
    mixed_img = tensor_to_image(mixed_img_tensor[0])  # Get the first mixed image

    # Draw the original image and the blended image
    plt.figure(figsize=(12, 6))

    # Original Image 1
    plt.subplot(1, 3, 1)
    plt.imshow(img1)
    plt.title("Original Image 1")
    plt.axis("off")

    # Original Image 2 (Contrast Enhanced)
    plt.subplot(1, 3, 2)
    plt.imshow(img2)
    plt.title("Enhanced Contrast Image 2")
    plt.axis("off")

    # Mixed pictures
    plt.subplot(1, 3, 3)
    plt.imshow(mixed_img)
    plt.title(f"Mixed Image (λ = {lam:.2f})")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# The path to the image in Google Drive
image_path = '/content/drive/My Drive/Eye_rgb/1144_left.jpg'

# Call the main function for Mixup visualization
visualize_mixup(image_path)